# 📁 Data Preprocessing & Split Creation

**Goal**: Create train/val/test splits and prepare data for training

**What this notebook does**:
1. Load image paths from EDA
2. Create stratified train/val/test splits
3. Copy/move files to split directories
4. Verify splits
5. Test data pipeline

**Author**: Your Name  
**Date**: YYYY-MM-DD

## Setup

In [1]:
import os
import shutil
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import yaml

# Set seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

print("✅ Imports complete")

✅ Imports complete


## 1. Load Configuration

In [2]:
# Load split ratios from config
with open('../configs/config.yaml', 'r') as f:
    config = yaml.safe_load(f)

TRAIN_RATIO = config['data']['train_split']  # 0.8
VAL_RATIO = config['data']['val_split']      # 0.1
TEST_RATIO = config['data']['test_split']    # 0.1

print(f"Split ratios: {TRAIN_RATIO}/{VAL_RATIO}/{TEST_RATIO}")

# Paths
RAW_DIR = Path('../data/raw')
PROCESSED_DIR = Path('../data/processed')

# Auto-detect emotion classes
EMOTIONS = sorted([d.name for d in RAW_DIR.iterdir() if d.is_dir()])
print(f"Emotions: {EMOTIONS}")

Split ratios: 0.8/0.1/0.1
Emotions: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


## 2. Load Image Paths

**Hint**: Use the CSV from EDA or scan directories again

In [3]:
# Option 1: Load from EDA results
df = pd.read_csv('../results/metrics/all_images.csv')
print(f"Loaded {len(df)} images from EDA results")

# Option 2: Scan directories (if CSV not available)
# TODO: Implement directory scanning if needed

print(f"\nClass distribution:")
print(df['emotion'].value_counts())

Loaded 49779 images from EDA results

Class distribution:
emotion
happy       11398
neutral      8166
sad          6535
angry        5920
disgust      5920
fear         5920
surprise     5920
Name: count, dtype: int64


## 3. Create Stratified Splits

**Key**: Use `stratify` parameter to maintain class distribution

In [6]:
# Split: train + (val+test)
train_df, temp_df = train_test_split(
    df,
    test_size=(VAL_RATIO + TEST_RATIO),
    stratify=df['emotion'],
    random_state=RANDOM_SEED
)

# Split: val + test
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,  # TODO: Replace with calculated value
    stratify=temp_df['emotion'],
    random_state=RANDOM_SEED
)

print(f"Train: {len(train_df)} images")
print(f"Val:   {len(val_df)} images")
print(f"Test:  {len(test_df)} images")
print(f"Total: {len(train_df) + len(val_df) + len(test_df)} images")

Train: 39823 images
Val:   4978 images
Test:  4978 images
Total: 49779 images


## 4. Verify Split Distribution

**Check**: Each emotion should have same ratio in all splits

In [7]:
# Create verification DataFrame
verification = pd.DataFrame({
    'train': train_df['emotion'].value_counts().sort_index(),
    'val': val_df['emotion'].value_counts().sort_index(),
    'test': test_df['emotion'].value_counts().sort_index()
}).fillna(0).astype(int)

verification['total'] = verification.sum(axis=1)
verification['train_%'] = (verification['train'] / verification['total'] * 100).round(1)
verification['val_%'] = (verification['val'] / verification['total'] * 100).round(1)
verification['test_%'] = (verification['test'] / verification['total'] * 100).round(1)

print(verification)

# TODO: Add assertions to verify splits are within acceptable range
# Example: assert all train percentages are between 79% and 81%

          train   val  test  total  train_%  val_%  test_%
emotion                                                   
angry      4736   592   592   5920     80.0   10.0    10.0
disgust    4736   592   592   5920     80.0   10.0    10.0
fear       4736   592   592   5920     80.0   10.0    10.0
happy      9118  1140  1140  11398     80.0   10.0    10.0
neutral    6533   816   817   8166     80.0   10.0    10.0
sad        5228   654   653   6535     80.0   10.0    10.0
surprise   4736   592   592   5920     80.0   10.0    10.0


## 5. Create Directory Structure

In [8]:
# Create processed data directories
for split in ['train', 'val', 'test']:
    for emotion in EMOTIONS:
        dir_path = PROCESSED_DIR / split / emotion
        dir_path.mkdir(parents=True, exist_ok=True)

print("✅ Directory structure created")
print(f"\nStructure:")
print(f"processed/")
print(f"├── train/{EMOTIONS[0]}/")
print(f"├── val/{EMOTIONS[0]}/")
print(f"└── test/{EMOTIONS[0]}/")

✅ Directory structure created

Structure:
processed/
├── train/angry/
├── val/angry/
└── test/angry/


## 6. Copy Files to Splits

**Choice**: Copy (safe) or Move (saves space)

**Warning**: Moving will delete files from raw directory!

In [9]:
# Choose operation: 'copy' or 'move'
OPERATION = 'copy'  # TODO: Change to 'move' if you want to move files

def transfer_files(df, split_name):
    """Copy or move files to split directory"""
    print(f"\nProcessing {split_name} set...")
    
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        src = Path(row['path'])
        dst = PROCESSED_DIR / split_name / row['emotion'] / src.name
        
        # TODO: Implement copy or move logic
        if OPERATION == 'copy':
            shutil.copy2(src, dst)
        elif OPERATION == 'move':
            shutil.move(src, dst)
        # TODO: Add error handling

# Execute transfer
transfer_files(train_df, 'train')
transfer_files(val_df, 'val')
transfer_files(test_df, 'test')

print("\n✅ File transfer complete!")


Processing train set...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 39823/39823 [00:23<00:00, 1719.84it/s]



Processing val set...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4978/4978 [00:02<00:00, 1683.17it/s]



Processing test set...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4978/4978 [00:02<00:00, 1757.09it/s]


✅ File transfer complete!


## 7. Verify File Counts

In [10]:
# Count files in each split
for split in ['train', 'val', 'test']:
    print(f"\n{split.upper()}:")
    total = 0
    for emotion in EMOTIONS:
        path = PROCESSED_DIR / split / emotion
        count = len(list(path.glob('*.*')))
        total += count
        print(f"  {emotion}: {count}")
    print(f"  TOTAL: {total}")

# TODO: Add assertions to verify counts match expected


TRAIN:
  angry: 4736
  disgust: 4736
  fear: 4736
  happy: 9118
  neutral: 6533
  sad: 5228
  surprise: 4736
  TOTAL: 39823

VAL:
  angry: 592
  disgust: 592
  fear: 592
  happy: 1140
  neutral: 816
  sad: 654
  surprise: 592
  TOTAL: 4978

TEST:
  angry: 592
  disgust: 592
  fear: 592
  happy: 1140
  neutral: 817
  sad: 653
  surprise: 592
  TOTAL: 4978


## 8. Save Split Information

In [ ]:
# Save split DataFrames for reference
train_df.to_csv('../results/metrics/train_split.csv', index=False)
val_df.to_csv('../results/metrics/val_split.csv', index=False)
test_df.to_csv('../results/metrics/test_split.csv', index=False)

# Save split info
split_info = {
    'random_seed': RANDOM_SEED,
    'train_ratio': TRAIN_RATIO,
    'val_ratio': VAL_RATIO,
    'test_ratio': TEST_RATIO,
    'train_count': len(train_df),
    'val_count': len(val_df),
    'test_count': len(test_df),
    'operation': OPERATION
}

with open('../results/metrics/split_info.yaml', 'w') as f:
    yaml.dump(split_info, f)

print("✅ Split information saved")

## 9. Test Data Loading (PyTorch)

**Quick test**: Verify we can load images with PyTorch

In [11]:
# TODO: Implement basic PyTorch data loading test
# Hints:
# 1. Import torchvision.transforms
# 2. Create ImageFolder dataset
# 3. Create DataLoader
# 4. Load one batch
# 5. Print shapes

try:
    from torchvision import datasets, transforms
    from torch.utils.data import DataLoader
    
    # Basic transforms
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])
    
    # Load train set
    train_dataset = datasets.ImageFolder(
        root=str(PROCESSED_DIR / 'train'),
        transform=transform
    )
    
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    # Test loading
    images, labels = next(iter(train_loader))
    print(f"✅ Data loading test passed")
    print(f"Batch shape: {images.shape}")
    print(f"Labels shape: {labels.shape}")
    print(f"Class mapping: {train_dataset.class_to_idx}")
    
except Exception as e:
    print(f"❌ Data loading test failed: {e}")

✅ Data loading test passed
Batch shape: torch.Size([32, 3, 224, 224])
Labels shape: torch.Size([32])
Class mapping: {'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}


## Summary

✅ **Completed**:
- Loaded image paths
- Created stratified splits
- Verified distributions
- Copied/moved files
- Tested data loading

📂 **Output Structure**:
```
data/processed/
├── train/
│   ├── angry/
│   ├── happy/
│   └── ...
├── val/
└── test/
```

🎯 **Next Steps**:
1. Run baseline model training
2. Evaluate on validation set
3. Iterate and improve